In [1]:
import pandas as pd
import os
from dotenv import load_dotenv
import duckdb
load_dotenv()

True

In [2]:
product_sales_parquet = os.getenv('product_sales_parquet')
file_path = os.getenv('file_path')

In [3]:
df = duckdb.read_parquet(product_sales_parquet).df()

In [4]:
df.head()

,order_id,customer_id,product,price_each,qty_ordered,total_price,order_time_stamp,order_date_part,year,month,...,day_name_num,day_month,purchase_address,city,state_name,capital,latitude,longitude,store,payment_method
0,147268,125160,Wired Headphones,11.99,1,11.99,2019-01-01 03:07:00,2019-01-01,2019,1,...,2,1,"9 Lake St, New York City, NY 10001",New York City,New York,Albany,35.780430,-78.639099,Walmart,Credit
1,148041,103419,USB-C Charging Cable,11.95,1,11.95,2019-01-01 03:40:00,2019-01-01,2019,1,...,2,1,"760 Church St, San Francisco, CA 94016",San Francisco,California,Sacramento,38.576668,-121.493629,eBay,Credit
2,149343,99418,Apple Airpods Headphones,150.00,1,150.00,2019-01-01 04:56:00,2019-01-01,2019,1,...,2,1,"735 5th St, New York City, NY 10001",New York City,New York,Albany,35.780430,-78.639099,eBay,Credit
3,149964,101715,AAA Batteries (4-pack),2.99,1,2.99,2019-01-01 05:53:00,2019-01-01,2019,1,...,2,1,"75 Jackson St, Dallas, TX 75001",Dallas,Texas,Austin,30.274670,-97.740349,eBay,Credit
4,149350,132005,USB-C Charging Cable,11.95,2,23.90,2019-01-01 06:03:00,2019-01-01,2019,1,...,2,1,"943 2nd St, Atlanta, GA 30301",Atlanta,Georgia,Atlanta,30.438118,-84.281296,Target,Cash


## Daily Sales and Orders

In [5]:
daily_sales_orders = duckdb.sql(
    """
    select
    day_name_num+1 as day_name_num,
    day_name,
    sum(qty_ordered) as total_orders,
    sum(total_price) as total_sales
    from df
    group by day_name_num, day_name
    order by day_name_num asc
    """
).df()

In [6]:
daily_sales_orders

,day_name_num,day_name,total_orders,total_sales
0,1,Sun,29745.0,4927249.40
1,2,Mon,29860.0,4877588.21
2,3,Tue,30686.0,5086275.40
3,4,Wed,29718.0,4978153.07
4,5,Thu,29618.0,4833763.73
5,6,Fri,29415.0,4853642.26
6,7,Sat,29729.0,4900195.58


In [7]:
filename= 'daily_sales_orders.parquet'
parquet = file_path + '\\' + filename
daily_sales_orders.to_parquet(parquet,compression='gzip')

## Quarterly Orders and Sales

In [7]:
quarter_sales_orders = duckdb.execute(
    """
    select
    (case
        when month_name in ('Jan','Feb','Mar') then 'Q1'
        when month_name in ('Apr','May','Jun') then 'Q2'
        when month_name in ('Jul','Aug','Sep') then 'Q3'
        when month_name in ('Oct','Nov','Dec') then 'Q4'
    end) as quarter,
    sum(total_price) as total_sales,
    sum(qty_ordered) as total_orders
    from df
    group by quarter
    order by quarter asc
    """
).df()

In [8]:
quarter_sales_orders.head()

,quarter,total_sales,total_orders
0,Q1,6.817794e+06,41262.0
1,Q2,9.116114e+06,54423.0
2,Q3,6.982010e+06,42574.0
3,Q4,1.154095e+07,70512.0


In [9]:
filename= 'quarter_sales_orders.parquet'
parquet = file_path + '\\' + filename
quarter_sales_orders.to_parquet(parquet,compression='gzip')

## Products Sales and Orders Percentage

In [9]:
products_sales_orders_percentage = duckdb.execute(
"""
with q1 as (
    select
    sum(total_price) as total_sales,
    sum(qty_ordered) as total_orders
    from df
),
    q2 as (
    select
    product,
    sum(qty_ordered) as total_orders,
    sum(total_price) as total_sales
    from df
    group by product
    order by total_orders desc
),
    q3 as (
    select
    product,
    total_orders,
    total_sales,
    total_orders/(select total_orders from q1) as order_percentage,
    total_sales/(select total_sales from q1) as sale_percentage
    from q2
    group by product,order_percentage,sale_percentage,total_orders,total_sales
    order by total_orders desc   
)

select
*
from q3
"""
).df()

In [11]:
products_sales_orders_percentage.head()

,product,total_orders,total_sales,order_percentage,sale_percentage
0,AAA Batteries (4-pack),30981.0,92633.19,0.148397,0.002688
1,AA Batteries (4-pack),27615.0,106041.60,0.132274,0.003078
2,USB-C Charging Cable,23927.0,285927.65,0.114609,0.008298
3,Lightning Charging Cable,23163.0,346286.85,0.110949,0.010050
4,Wired Headphones,20520.0,246034.80,0.098290,0.007140


In [26]:
filename= 'products_sales_orders_percentage.parquet'
parquet = r'C:\Users\USER\Desktop\elt_test_run\md_product_sales\powerbi\parquet_files\products' + '\\' + filename
products_sales_orders_percentage.to_parquet(parquet,compression='gzip')

## Heatmap of product sales and orders per month

In [14]:
heatmap_products_sales_orders = duckdb.execute(
    """
    select
	product,
	month_name,
	month,
	day_name,
	day_name_num+1 as day_name_num,
	sum(total_price) as total_sales,
	sum(qty_ordered)::int as total_orders
	from df
	group by product, month_name, day_name, month, day_name_num
	order by product asc, month asc, day_name_num asc

    """
).df()

In [15]:
heatmap_products_sales_orders.head()

,product,month_name,month,day_name,day_name_num,total_sales,total_orders
0,20in Monitor,Jan,1,Sun,1,2859.74,26
1,20in Monitor,Jan,1,Mon,2,2859.74,26
2,20in Monitor,Jan,1,Tue,3,2859.74,26
3,20in Monitor,Jan,1,Wed,4,5059.54,46
4,20in Monitor,Jan,1,Thu,5,3079.72,28


In [14]:
filename= 'heatmap_products_sales_orders.parquet'
parquet = r'C:\Users\USER\Desktop\elt_test_run\md_product_sales\powerbi\parquet_files\products' + '\\' + filename
heatmap_products_sales_orders.to_parquet(parquet,compression='gzip')

## Sales of products per payment method

In [16]:
sales_per_method = duckdb.execute(
    """
    select
    product,
    sum(total_price) as total_sales,
    payment_method
    from df
    group by product, payment_method
    order by product asc
    """
).df()
sales_per_method.head()

,product,total_sales,payment_method
0,20in Monitor,231528.95,Cash
1,20in Monitor,221959.82,Credit
2,27in 4K Gaming Monitor,1201949.18,Cash
3,27in 4K Gaming Monitor,1230808.44,Credit
4,27in FHD Monitor,564562.36,Credit


## Sales of products per payment method (Pivot)

In [19]:
sales_per_method_pivot = duckdb.execute(
    """
    select
    product,
    sum(total_price) as total_sales,
    sum(case when payment_method = 'Cash' then total_price else 0 end) as 'cash_sales',
    sum(case when payment_method = 'Credit' then total_price else 0 end) as 'credit_sales'
    --sum(total_price) filter (where payment_method = 'Cash') as cash_sales,
    --sum(total_price) filter (where payment_method = 'Credit') as credit_sales
    from df
    group by product
    order by lower(product) asc
    """
).df()

In [20]:
sales_per_method_pivot.head()

,product,total_sales,cash_sales,credit_sales
0,20in Monitor,453488.77,231528.95,221959.82
1,27in 4K Gaming Monitor,2432757.62,1201949.18,1230808.44
2,27in FHD Monitor,1130624.62,566062.26,564562.36
3,34in Ultrawide Monitor,2352898.08,1162769.40,1190128.68
4,AA Batteries (4-pack),106041.60,52934.40,53107.20


In [21]:
filename= 'heatmap_sales_per_method_pivot.parquet'
parquet = r'C:\Users\USER\Desktop\elt_test_run\md_product_sales\powerbi\parquet_files\products' + '\\' + filename
sales_per_method_pivot.to_parquet(parquet,compression='gzip')